# Simplified `GasGenerator`

In [ ]:
import numpy as np

from cosapp.drivers import NonLinearSolver
from cosapp.recorders import DataFrameRecorder

from pyturbo.systems import GasGeneratorSimple

In [ ]:
gg = GasGeneratorSimple('gg')
solver = gg.add_driver(NonLinearSolver('solver', factor=0.4))

solver.add_unknown('fl_in.W').add_equation("turbine.shaft_out.power == 1e6").add_unknown("combustor.fuel_in.W");

In [ ]:
gg.fuel_in.W = 0.05
gg.fl_in.W = 5
gg.fl_in.pt = 1e5
gg.fl_in.Tt = 300.
gg.turbine.aero.er = 2.
gg.turbine.aero.Ncdes = 30

gg.geom.scale = 0.48
gg.geom.kp.inlet_hub = np.r_[0.4, 0.0]
gg.geom.kp.inlet_tip = np.r_[0.5, 0.0]
gg.geom.kp.exit_hub = np.r_[0.45, 1.0]
gg.geom.kp.exit_tip = np.r_[0.5, 1.0]

gg.compressor.stage_count = 10
gg.turbine.stage_count = 2

gg.run_drivers()

print('mass flow : ', gg.fl_out.W, 'kg/s')
print('combustion temp : ', gg.combustor.fl_out.Tt, 'K')
print('fuel flow : ', gg.combustor.fuel_in.W, 'kg/s')
print('shaft power : ', gg.turbine.shaft_out.power / 1e6, 'MW')
print('shaft rotational speed : ', gg.turbine.shaft_out.N, 'rpm')
print('compressor pressure ratio : ', gg.compressor.aero.pr)
print('compressor loading : ', gg.compressor.aero.psi)
print('turbine expansion ratio : ', gg.turbine.aero.er)
print('turbine loading : ', gg.turbine.aero.psi)

In [ ]:
gg.compressor.aero.psi, gg.turbine.aero.psi

In [ ]:
gg.drivers.clear()

design = gg.add_driver(NonLinearSolver('design', factor=0.2))
design.add_unknown('fl_in.W').add_unknown('turbine.aero.er')
design.runner.add_target("turbine.shaft_out.power").add_unknown("combustor.fuel_in.W")
design.extend(gg.design_methods["sizing"])

rec = design.runner.add_recorder(DataFrameRecorder(includes=['*'], hold=False))

gg.compressor.stage_count = 10
gg.turbine.stage_count = 2
gg.compressor.aero.pr = 22.
gg.compressor.aero.psi = 0.4
gg.turbine.shaft_out.power = 10e6
gg.turbine.aero.fl_in.Tt = 1700.
gg.run_drivers()

In [ ]:
gg.fl_in.pt = 2.5 * 101325.
gg.fl_in.Tt = 500.
gg.turbine.shaft_out.power = 15e6
gg.run_drivers()

In [ ]:
print('mass flow : ', gg.fl_out.W, 'kg/s')
print('combustion temp : ', gg.combustor.fl_out.Tt, 'K')
print('fuel flow : ', gg.combustor.fuel_in.W, 'kg/s')
print('shaft power : ', gg.turbine.shaft_out.power / 1e6, 'MW')
print('shaft rotational speed : ', gg.turbine.shaft_out.N, 'rpm')
print('compressor pressure ratio : ', gg.compressor.aero.pr)
print('compressor loading : ', gg.compressor.aero.psi)
print('turbine expansion ratio : ', gg.turbine.aero.er)
print('turbine loading : ', gg.turbine.aero.psi)

In [ ]:
gg.jupyter_view()

In [ ]:
gg.turbine.shaft_out.power = 30e6
gg.run_drivers()

gg.update_jupyter_view()